In [2]:
pip install segmentation-models-pytorch

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.3/121.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
%load_ext tensorboard
import os
import sys
import cv2
import random
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from google.colab.patches import cv2_imshow
import sklearn
from sklearn import model_selection
from sklearn import metrics

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
import torch.nn as nn
from collections import defaultdict
import torchvision
from torchvision import transforms
import torch.nn.functional as F
from tqdm.auto import tqdm
from torch.utils.tensorboard import SummaryWriter
import copy
import time

import segmentation_models_pytorch

import albumentations as A
#from albumentations.pytorch import ToTensorV2
from albumentations.pytorch.transforms import ToTensorV2

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [10]:
os.makedirs("/content/new/Train/Labeled/Flooded/image", exist_ok=True)
os.makedirs("/content/new/Train/Labeled/Non-Flooded/image", exist_ok=True)
os.makedirs("/content/new/Train/Labeled/Flooded/mask", exist_ok=True)
os.makedirs("/content/new/Train/Labeled/Non-Flooded/mask", exist_ok=True)
os.makedirs("/content/new/Train/Unlabeled/image", exist_ok=True)
os.makedirs("/content/new/Test/image", exist_ok=True)

In [11]:
# !ls /content/drive/MyDrive/FloodNet\ Challenge\ @\ EARTHVISION\ 2021\ -\ Track\ 1/Train/Labeled

!ls "/content/drive/MyDrive/FloodNet Challenge @ EARTHVISION 2021 - Track 1/Train/Labeled"

Flooded  Non-Flooded


In [12]:

!ls "/content/new/Train/Labeled/Flooded"

image  mask


In [13]:
RESIZE=(512,512)
temp_root = "/content/drive/MyDrive/FloodNet Challenge @ EARTHVISION 2021 - Track 1"
local_root = "/content/new"
import cv2
import os
from tqdm import tqdm

# Your resize and save function
def resize_and_save(path, resize=(256, 256), samples=None):
    # Assuming temp_root and local_root are defined elsewhere in your code
    for img_name in tqdm(os.listdir(os.path.join(temp_root, path))[:samples]):
        img_path = os.path.join(temp_root, path, img_name)

        # Check if the file exists
        if os.path.exists(img_path):
            # Read the image
            img = cv2.imread(img_path)

            # Check if image is successfully loaded
            if img is not None:
                # Resize the image
                img_resized = cv2.resize(img, resize)

                # Save the resized image
                save_path = os.path.join(local_root, path, img_name)
                cv2.imwrite(save_path, img_resized)
            else:
                print(f"Error reading image: {img_path}")
        else:
            print(f"File does not exist: {img_path}")


In [14]:
resize_and_save("Train/Labeled/Flooded/image")
resize_and_save("Train/Labeled/Non-Flooded/image")
resize_and_save("Train/Labeled/Flooded/mask")
resize_and_save("Train/Labeled/Non-Flooded/mask")
resize_and_save("Train/Unlabeled/image")
resize_and_save("Test/image")
resize_and_save("Train/Unlabeled/image", samples=100)

100%|██████████| 100/100 [00:23<00:00,  4.23it/s]


In [15]:
flood_dir = "Train/Labeled/Flooded"
non_flood_dir = "Train/Labeled/Non-Flooded"

f_img_dir = os.path.join(local_root, flood_dir, "image")
f_mask_dir = os.path.join(local_root, flood_dir, "mask")
n_img_dir = os.path.join(local_root, non_flood_dir, "image")
n_mask_dir = os.path.join(local_root, non_flood_dir, "mask")

f_img = len(os.listdir(f_img_dir))
f_mask = len(os.listdir(f_mask_dir))
n_img = len(os.listdir(n_img_dir))
n_mask = len(os.listdir(n_mask_dir))
print(f"Flooded images:      {f_img} Flooded masks:     {f_mask}")
print(f"Non-Flooded images: {n_img} Non-Flooded masks: {n_mask}")

Flooded images:      51 Flooded masks:     51
Non-Flooded images: 347 Non-Flooded masks: 347


In [16]:
CLASSES={'Background':0,'Building-flooded':1,'Building-non-flooded':2,'Road-flooded':3,'Road-non-flooded':4,
         'Water':5,'Tree':6,'Vehicle':7,'Pool':8,'Grass':9}
IMG_DIM= 512

In [17]:
TENSORBOARD_DIR = "/content/drive/MyDrive/runs_h/"
MODEL_DIR = "/content/drive/MyDrive/models_h/"
UNLABELLED_SPLIT = 100

os.makedirs(TENSORBOARD_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

ENCODER_DEPTH=5
DECODER_CHANNELS=256
BATCH_SIZE= [8]
EPOCHS= 100
LR = [1e-3]
ENCODER_NAME= 'efficientnet-b3'

In [19]:
dirs = """
flood_dir
non_flood_dir

f_img_dir
f_mask_dir
n_img_dir
n_mask_dir
"""
x_f = [os.path.join(f_img_dir, file) for file in sorted(os.listdir(f_img_dir))]
x_n = [os.path.join(n_img_dir, file) for file in sorted(os.listdir(n_img_dir))]

x = x_f + x_n

x_train, x_test = model_selection.train_test_split(x, test_size=0.3, shuffle=True)

train_transform1 = A.Compose([
        A.Resize(IMG_DIM, IMG_DIM),
        A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=30, p=0.5),
        # A.RGBShift(r_shift_limit=25, g_shift_limit=25, b_shift_limit=25, p=0.5),
        A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
    ])

train_transform2=A.Compose([ToTensorV2()])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [57]:
class SegDataset:
    def __init__(self, x_paths, trans1, trans2, img_dim, unlabelled=False):
        self.x_paths = x_paths
        self.unlabelled = unlabelled
        if not self.unlabelled:
            self.y_paths = [x.replace("image", "mask").replace(".jpg", "_lab.png") for x in self.x_paths]
        else:
            self.y_paths = None
        self.img_dim = img_dim
        self.trans1 = trans1
        self.trans2 = trans2

    def __len__(self):
        return len(self.x_paths)

    def get_newMask(self, mask, classes, dim=IMG_DIM):
        if mask.ndim == 3:
            mask = torch.as_tensor(mask[:,:,0], dtype=torch.int64)
        elif mask.ndim == 2:
            mask = torch.as_tensor(mask, dtype=torch.int64)
        return torch.moveaxis(torch.nn.functional.one_hot(mask, num_classes=len(classes)), -1, 0)

    def __getitem__(self, index):
        # Read and convert image
        image = cv2.imread(self.x_paths[index])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Preprocess the image (keep this consistent)
        kernel = np.ones((2, 2), np.uint8)
        image = cv2.bilateralFilter(image, 5, 75, 75)
        image = cv2.erode(cv2.dilate(image, kernel, iterations=2), kernel, iterations=1)

        # Read and preprocess mask (if not unlabelled)
        if not self.unlabelled:
            mask = cv2.imread(self.y_paths[index], cv2.IMREAD_GRAYSCALE)
        else:
            mask = np.random.rand(512, 512, 3)  # Dummy mask for unlabelled data

        # Apply transformations to both image and mask
        # Apply first transformation (resize and augment) to both image and mask
        if self.trans1:
            transformed1 = self.trans1(image=image, mask=mask)
            image = transformed1["image"]
            mask = transformed1["mask"]

        # Apply second transformation (tensor conversion) to both image and mask
        if self.trans2:
            transformed2 = self.trans2(image=image, mask=mask)
            image = transformed2["image"]
            mask = transformed2["mask"]

        return image, mask


In [21]:
def dice_loss(pred, target, smooth = 1e-5):
    pred = pred.contiguous()
    target = target.contiguous()

    intersection = (pred * target).sum(dim=2).sum(dim=2)

    loss = (1 - ((2. * intersection + smooth) / (pred.sum(dim=2).sum(dim=2) + target.sum(dim=2).sum(dim=2) + smooth)))

    return loss.mean()

In [22]:
def calc_loss(pred, target, metrics, bce_weight=0.5, unlabelled=False):
    bce = F.binary_cross_entropy_with_logits(pred, target.to(torch.float32))
    pred = F.sigmoid(pred)
    dice = dice_loss(pred, target)

    target_np=target.data.cpu().numpy()
    pred_np=pred.data.cpu().numpy()
    MIoU= np.mean(sklearn.metrics.jaccard_score(np.argmax(target_np,axis=1).flatten(), np.argmax(pred_np,axis=1).flatten(), average=None))
    loss = bce * bce_weight + dice * (1 - bce_weight)
    if not unlabelled:
      metrics['bce'] += bce.data.cpu().numpy() * target.size(0)
      metrics['dice'] += dice.data.cpu().numpy() * target.size(0)
      metrics['loss'] += loss.data.cpu().numpy() * target.size(0)
      metrics['MIoU'] += MIoU * target.size(0)

    return loss

In [23]:
def print_metrics(metrics, epoch_samples, phase):
    outputs = []
    for k in metrics.keys():
        outputs.append("{}: {:4f}".format(k, metrics[k] / epoch_samples))
    print("{}: {}".format(phase, ", ".join(outputs)))

In [58]:
training_set = SegDataset(x_train,  train_transform1, train_transform2, img_dim=IMG_DIM)
testing_set = SegDataset(x_test, train_transform1,train_transform2, img_dim=IMG_DIM)
image_datasets = {'train': training_set, 'valid': testing_set}

u_dir = "/content/new/Train/Unlabeled/image"
unlabelled_paths = [os.path.join(u_dir, file) for file in os.listdir(u_dir)]
unlabelled_set = SegDataset(unlabelled_paths[0:-500], trans1=train_transform1,trans2=train_transform2, img_dim=IMG_DIM, unlabelled=True)
image_datasets["unlabelled"] = unlabelled_set

In [35]:
from torchvision import models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pretrained DeepLabV3 model
model = models.segmentation.deeplabv3_resnet101(pretrained=True)
model.classifier[4] = nn.Conv2d(256, len(CLASSES), kernel_size=(1, 1), stride=(1, 1))  # Adjust the output channels to the number of classes
model.to(device)

# Setup optimizer and learning rate scheduler
optimizer = optim.Adam(model.parameters(), lr=LR[0])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [36]:
def train_model(model, image_datasets, optimizer, scheduler, num_epochs=EPOCHS):
    best_model_wts = model.state_dict()
    best_miou = 0.0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch}/{num_epochs - 1}")
        print("-" * 10)


        for phase in ['train', 'valid', 'unlabelled']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            metrics = {'bce': 0.0, 'dice': 0.0, 'loss': 0.0, 'MIoU': 0.0}
            epoch_samples = 0
            for inputs, labels in tqdm(DataLoader(image_datasets[phase], batch_size=BATCH_SIZE[0], shuffle=True, num_workers=4)):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)['out']
                    loss = calc_loss(outputs, labels, metrics)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                epoch_samples += inputs.size(0)

            print_metrics(metrics, epoch_samples, phase)

            if phase == 'valid' or phase == 'unlabelled':
                miou = metrics['MIoU'] / epoch_samples
                if miou > best_miou:
                    best_miou = miou
                    best_model_wts = model.state_dict()

            if phase == 'train':
                scheduler.step()

        print()

    model.load_state_dict(best_model_wts)
    return model


In [ ]:
model = train_model(model, image_datasets, optimizer, scheduler, num_epochs=EPOCHS)

Epoch 0/99
----------


  0%|          | 0/35 [00:00<?, ?it/s]